In [ ]:
import boto3
import random

s3client = boto3.client('s3')

In [ ]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-website-' + str(random.randint(1, 9999999))
print bucket_name

In [ ]:
response = s3client.create_bucket(
    Bucket=bucket_name
)

print response

In [ ]:
# Set static web site configuration
response = s3client.put_bucket_website(
    Bucket=bucket_name,
    WebsiteConfiguration={
        'ErrorDocument': {
            'Key': 'error.html'
        },
        'IndexDocument': {
            'Suffix': 'index.html'
        }
    }
)

print response

In [ ]:
# Create index document
f = open("./index.html","w")
f.write("<html><head><title>Yo</title></head><body><h2>Yo, yo</h2></body></html>")
f.close()

In [ ]:
# Upload index document
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./index.html', bucket_name, 'index.html')
print response

In [ ]:
# Upload error document
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./error.html', bucket_name, 'error.html')
print response

In [ ]:
# Create error document
f = open("./error.html","w")
f.write("<html><head><title>Error Yo</title></head><body><h2>Error, yo</h2></body></html>")
f.close()

At this point you will get a forbidden back when you access the web site as nothing is readable by the world

In [ ]:
import os
region = os.environ['AWS_DEFAULT_REGION']
site = 'http://' + bucket_name + '.s3-website-' + region + '.amazonaws.com'

print site

https_proxy = os.environ['https_proxy']
print https_proxy
os.environ['http_proxy'] = https_proxy

In [ ]:
%%bash -s "$site"
curl $1

Now if we make the error page world readable then we get our error page back when trying to access index.html.
which is still not readable.

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'error.html')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
%%bash -s "$site"
curl $1

In [ ]:
# For grins we can make a signed url to allow it to be curled, this is just for playing
# around and has nothing to do with this example.
presigned_url = s3client.generate_presigned_url(
    'get_object', {'Bucket': bucket_name, 'Key': 'index.html'})

print presigned_url

In [ ]:
%%bash -s "$presigned_url"
curl $1

Let's make the index readable - curl now returns our expected page.

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'index.html')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
%%bash -s "$site"
curl $1

In [ ]:
# Delete our objects
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='index.html'
)

print response

response = s3client.delete_object(
        Bucket=bucket_name,
        Key='error.html'
)

print response

In [ ]:
response = s3client.delete_bucket(
    Bucket=bucket_name
)

print response